In [5]:
import requests
import yaml
from bs4 import BeautifulSoup

with open('./config/config.yml', 'r') as f:
    config = yaml.safe_load(f)
    
urlchotot = config['url']['urlchotot']
response = requests.get(url=urlchotot)
soup = BeautifulSoup(response.text, 'html.parser')
all_results = soup.find_all('div', attrs={'tabindex':"0"})
print(len(all_results))
for result in all_results:
    data = {}
    data["name"] = result.find('h3',class_="adonovt").text
    data["year"] = result.find('span', class_='c1u6gyxh').text
    data["price"] = result.find('span', class_='bfe6oav').text
    newRequest = requests.get("https://xe.chotot.com"+result.find('a',attrs={'itemprop':'item'}).get('href'))
    newSoup = BeautifulSoup(newRequest.text, 'html.parser')
    data["description"] = newSoup.find('p', attrs={'itemprop':'description'}).text
    data["urlcar"] = "https://xe.chotot.com"+result.find('a',attrs={'itemprop':'item'}).get('href')
    print(data)

200

25
{'name': 'THU - MUA GIÁ CAO TẤT CẢ CÁC DÒNG XE OTO', 'year': '2023', 'price': '50.000.000 đ', 'description': 'Siêu thị Ô tô Hà Nội thu mua tất cả các dòng xe của các nhãn hiệu xe: Ford, Kia, Mazda, Mitsubishi, Honda, Mitsubishi, Honda, Hyundai, Toyota, Chevrolet, Nissan, Mecerder, BMW, AUDI...\nTầm giá từ 300 triệu trở lên.\n\nCam kết:\n👉 Giá nào cũng mua, giá thu cao nhất.\n 👉 Bảo mật thông tin khách hàng 100%.\n 👉 Thanh toán 100% trong ngày.\n 👉 Hoa hồng cao cho người giới thiệu.\n 👉 Xem xe tận nơi, thu xe 24/7.\n 👉 Hỗ trợ mọi thủ tục, giấy tờ công chứng sang tên.\nCả nhà có xe ới em nhé ạ 😘😘\n___________________________________________\n𝐒𝐢ê𝐮 𝐓𝐡ị Ô 𝐓ô 𝐇à 𝐍ộ𝐢 - Thương hiệu xe lướt số 1 Việt Nam.\n🏣 389 Quốc Lộ 13, Hiệp Bình Phước, Thủ Đức, Hồ Chí Minh.\nCode: #22915352_01', 'urlcar': 'https://xe.chotot.com/mua-ban-oto-thanh-pho-thu-duc-tp-ho-chi-minh/117922237.htm#px=SR-stickyad-[PO-1][PL-top]'}
{'name': 'Chevrolet Cruze 2010 LTZ 1.8 . AT số tự động .', 'year': '2010', 'price': '11